## Assignments


### 1. Weather model

For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the **weatherinszeged** table from Thinkful's database.
* Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the *apparenttemperature* and the *temperature*. As explanatory variables, use *humidity* and *windspeed*. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why? 
* Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one? 
* Add *visibility* as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the *visibility* in terms of the improvement in the adjusted R-squared. Which one is more useful?
* Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [3]:
#Model 1 

Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed']]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:46:17   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In the table above, we see that for the first model, R-square is 0.288 and adjusted R-square is 0.288, indicating this model explains 28.8% of the variance in the target variable, leaving 71.2% unexplained, which is not satisfactory. 

In [4]:
#Model 2: adding interaction term to model 1

weather_df['hum_wind'] = weather_df['humidity'] * weather_df['windspeed']
Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed', 'hum_wind']]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:50:18   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0839      0.033      2.511      0.012       0.018       0.149
humidity       0.1775      0.043      4.133      0.000       0.093       0.262
windspeed      0.0905      0.002     36.797      0.000       0.086       0.095
hum_wind      -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-square and adjusted R-square of the second model is 0.341, which is higher than the previous model's. This improvement indicates that the interaction of humidity and windspeed explains some previously unexplained variance in the target variable. 

In [5]:
#Model 3: adding visibility to model 1

Y = weather_df['apparenttemperature'] - weather_df['temperature']
X = weather_df[['humidity', 'windspeed', 'visibility']]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:53:47   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-square and adjusted R-square of the third model is 0.304, which is higher than the previous model's. This improvement indicates that visibility explains some previously unexplained variance in the target variable. 

Comparing the models, we see that for the second model, the AIC is 333,400 and BIC is 333,400. For the third model, the AIC is 338,800 and BIC is 338,800. According to all of the metrics, I would choose the second model.

###  2. House prices model

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
* Do you think your model is satisfactory? If so, why?
* In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. 
* For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [6]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In the previous assignment, I chose 10 variables that have relative higher correlation with saleprice >=0.50. Also added categorical variables mszoning, street, and exterqual to the model where I saw variance between groups. 

In [8]:
subcat = ['mszoning', 'street', 'exterqual']

for i in subcat:
    dummy_column_names = []
    house_df = pd.concat([house_df,
                         pd.get_dummies(house_df[i], prefix=i, drop_first=True)], axis=1
                        )
    
house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,salecondition,saleprice,mszoning_FV,mszoning_RH,mszoning_RL,mszoning_RM,street_Pave,exterqual_Fa,exterqual_Gd,exterqual_TA
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,Normal,208500,0,0,1,0,1,0,1,0
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,Normal,181500,0,0,1,0,1,0,0,1
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,Normal,223500,0,0,1,0,1,0,1,0
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,Abnorml,140000,0,0,1,0,1,0,0,1
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,Normal,250000,0,0,1,0,1,0,1,0


In [10]:
#Model 1

cat_column_names = list(house_df.columns[-8:])

Y = house_df['saleprice']

X = house_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'firstflrsf',
              'fullbath', 'totrmsabvgrd', 'yearbuilt', 'yearremodadd'] + cat_column_names]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()

lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     309.5
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:08:48   Log-Likelihood:                -17389.
No. Observations:                1460   AIC:                         3.482e+04
Df Residuals:                    1441   BIC:                         3.492e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.281e+05   1.39e+05     -5.961      0.000    -1.1e+06   -5.56e+05
overallqual   1.637e+04   1237.233     13.234      0.000    1.39e+04    1.88e+04
grlivarea       53.8024      4.077     13.198      0.000      45.806      61.799
garagecars    1.206e+04   2936.522      4.107      0.000    6300.127    1.78e+04
garagearea       8.8562      9.975      0.888      0.375     -10.710      28.423
totalbsmtsf     16.9508      4.128      4.106      0.000       8.852      25.049
firstflrsf       7.0145      4.825      1.454      0.146      -2.450      16.479
fullbath     -5485.6729   2587.432     -2.120      0.034   -1.06e+04    -410.136
totrmsabvgrd -1431.8145   1089.432     -1.314      0.189   -3568.857     705.228
yearbuilt      150.9792     52.626      2.869      0.004      47.747     254.211
yearremodadd   267.7773     62.954      4.254      0.000     144.286     391.268
mszoning_FV   2.112e+04   1.33e+04      1.589      0.112   -4957.174    4.72e+04
mszoning_RH   1.709e+04   1.51e+04      1.134      0.257   -1.25e+04    4.67e+04
mszoning_RL   3.003e+04   1.23e+04      2.436      0.015    5844.690    5.42e+04
mszoning_RM   1.283e+04   1.24e+04      1.037      0.300   -1.14e+04    3.71e+04
street_Pave    494.6105   1.61e+04      0.031      0.975   -3.11e+04    3.21e+04
exterqual_Fa -5.767e+04   1.29e+04     -4.457      0.000   -8.31e+04   -3.23e+04
exterqual_Gd   -5.8e+04   5716.585    -10.146      0.000   -6.92e+04   -4.68e+04
exterqual_TA -6.831e+04   6487.488    -10.530      0.000    -8.1e+04   -5.56e+04
==============================================================================
Omnibus:                      644.161   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            92540.270
Skew:                          -0.988   Prob(JB):                         0.00
Kurtosis:                      41.953   Cond. No.                     5.29e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.29e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In the table above, we see that for the first model, R-square is 0.795 and adjusted R-square is 0.792, AIC is 34,820 and BIC is 34,920. According to the model, except for garagearea, firstflrsf, totrmsabvgrd, mszoning, and street, the regression coefficient for other features are statistically significant. I will exclude these insignificant features from the new model. 

In [11]:
#Model 2

Y = house_df['saleprice']

X = house_df[['overallqual', 'grlivarea', 'garagecars',
              'fullbath', 'yearbuilt', 'yearremodadd', 'mszoning_RL']]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()

lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     664.4
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:11:30   Log-Likelihood:                -17496.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1452   BIC:                         3.505e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.184e+06   1.31e+05     -9.021      0.000   -1.44e+06   -9.27e+05
overallqual   2.258e+04   1184.126     19.071      0.000    2.03e+04    2.49e+04
grlivarea       59.3148      2.979     19.910      0.000      53.471      65.159
garagecars    1.587e+04   1842.753      8.610      0.000    1.23e+04    1.95e+04
fullbath     -9395.5710   2687.441     -3.496      0.000   -1.47e+04   -4123.889
yearbuilt      260.0664     52.302      4.972      0.000     157.471     362.662
yearremodadd   300.8563     65.392      4.601      0.000     172.584     429.129
mszoning_RL     1.8e+04   2627.373      6.852      0.000    1.28e+04    2.32e+04
==============================================================================
Omnibus:                      476.027   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17945.068
Skew:                           0.817   Prob(JB):                         0.00
Kurtosis:                      20.097   Cond. No.                     4.12e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.12e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

By fitting the second model, R-squared and Adjusted R-squared decreased to 0.762 and 0.761, AIC and BIC increased to 35,010 and 35,050. Since the second model is worse than the first model, I fitted the third and fourth models to the data by adding 1 more feature to the first model. 

In [20]:
#Model 3: adding fireplaces to model 1

cat_column_names = list(house_df.columns[-8:])

Y = house_df['saleprice']

X = house_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'firstflrsf',
              'fullbath', 'totrmsabvgrd', 'yearbuilt', 'yearremodadd', 'fireplaces'] + cat_column_names]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()

lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.798
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     299.9
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:20:42   Log-Likelihood:                -17375.
No. Observations:                1460   AIC:                         3.479e+04
Df Residuals:                    1440   BIC:                         3.490e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.869e+05   1.38e+05     -6.419      0.000   -1.16e+06   -6.16e+05
overallqual   1.525e+04   1245.494     12.240      0.000    1.28e+04    1.77e+04
grlivarea       48.8535      4.152     11.767      0.000      40.709      56.998
garagecars     1.05e+04   2926.172      3.587      0.000    4755.584    1.62e+04
garagearea      14.1490      9.939      1.424      0.155      -5.348      33.646
totalbsmtsf     17.7055      4.095      4.324      0.000       9.674      25.737
firstflrsf       4.1055      4.815      0.853      0.394      -5.340      13.551
fullbath     -4323.1175   2574.339     -1.679      0.093   -9372.974     726.739
totrmsabvgrd  -902.3378   1084.619     -0.832      0.406   -3029.941    1225.265
yearbuilt      156.6580     52.172      3.003      0.003      54.317     259.000
yearremodadd   294.5443     62.610      4.704      0.000     171.727     417.362
fireplaces    9115.7443   1759.531      5.181      0.000    5664.225    1.26e+04
mszoning_FV   2.091e+04   1.32e+04      1.587      0.113   -4938.333    4.68e+04
mszoning_RH   1.847e+04   1.49e+04      1.236      0.217   -1.08e+04    4.78e+04
mszoning_RL   2.757e+04   1.22e+04      2.254      0.024    3577.934    5.16e+04
mszoning_RM   1.221e+04   1.23e+04      0.995      0.320   -1.19e+04    3.63e+04
street_Pave   3869.3693    1.6e+04      0.242      0.809   -2.75e+04    3.52e+04
exterqual_Fa -5.782e+04   1.28e+04     -4.508      0.000    -8.3e+04   -3.27e+04
exterqual_Gd  -5.83e+04   5666.294    -10.288      0.000   -6.94e+04   -4.72e+04
exterqual_TA -6.906e+04   6431.716    -10.737      0.000   -8.17e+04   -5.64e+04
==============================================================================
Omnibus:                      620.321   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            95936.558
Skew:                          -0.886   Prob(JB):                         0.00
Kurtosis:                      42.672   Cond. No.                     5.30e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.3e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
#Model 4: adding lotarea to model 1

cat_column_names = list(house_df.columns[-8:])

Y = house_df['saleprice']

X = house_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'firstflrsf',
              'fullbath', 'totrmsabvgrd', 'yearbuilt', 'yearremodadd', 'lotarea'] + cat_column_names]

X = sm.add_constant(X)

lin_reg = sm.OLS(Y, X).fit()

lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     302.3
Date:                Fri, 18 Oct 2019   Prob (F-statistic):               0.00
Time:                        12:21:17   Log-Likelihood:                -17371.
No. Observations:                1460   AIC:                         3.478e+04
Df Residuals:                    1440   BIC:                         3.489e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.942e+05   1.38e+05     -6.494      0.000   -1.16e+06   -6.24e+05
overallqual   1.683e+04   1224.652     13.741      0.000    1.44e+04    1.92e+04
grlivarea       50.4347      4.066     12.404      0.000      42.459      58.411
garagecars    1.164e+04   2901.968      4.013      0.000    5952.073    1.73e+04
garagearea       8.4494      9.855      0.857      0.391     -10.882      27.781
totalbsmtsf     14.9630      4.092      3.657      0.000       6.936      22.990
firstflrsf       5.7599      4.771      1.207      0.228      -3.600      15.119
fullbath     -5526.0694   2556.272     -2.162      0.031   -1.05e+04    -511.653
totrmsabvgrd -1094.8212   1077.759     -1.016      0.310   -3208.967    1019.324
yearbuilt      175.4778     52.151      3.365      0.001      73.178     277.777
yearremodadd   266.7670     62.196      4.289      0.000     144.763     388.771
lotarea          0.6299      0.104      6.029      0.000       0.425       0.835
mszoning_FV   1.784e+04   1.31e+04      1.357      0.175   -7950.003    4.36e+04
mszoning_RH    1.44e+04   1.49e+04      0.967      0.334   -1.48e+04    4.36e+04
mszoning_RL   2.533e+04   1.22e+04      2.075      0.038    1383.000    4.93e+04
mszoning_RM   1.083e+04   1.22e+04      0.886      0.376   -1.32e+04    3.48e+04
street_Pave   2.313e+04   1.63e+04      1.415      0.157   -8926.576    5.52e+04
exterqual_Fa -5.442e+04   1.28e+04     -4.253      0.000   -7.95e+04   -2.93e+04
exterqual_Gd -5.819e+04   5647.813    -10.304      0.000   -6.93e+04   -4.71e+04
exterqual_TA -6.919e+04   6411.012    -10.793      0.000   -8.18e+04   -5.66e+04
==============================================================================
Omnibus:                      714.352   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           115749.124
Skew:                          -1.192   Prob(JB):                         0.00
Kurtosis:                      46.555   Cond. No.                     2.17e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In the tables above, we see that for the third model, R-squared is 0.798, adjusted R-square is 0.796, AIC is 34,790 and BIC is 34,900. For the fourth model, R-squared is 0.800, adjusted R-square is 0.797, AIC is 34,780 and BIC is 34,890. According to all the metrics, the fourth model performs the best. 